# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [37]:
!wget https://norvig.com/big.txt

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [38]:
import re
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

import nltk
import string
import random as rd

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
bigrams = []
dictionary_prediction = {}

with open("bigrams.txt", errors='replace') as f:
    for line in f:
        bigrams.append(line.strip())


for i in range(len(bigrams) - 1):
    bigrams[i] = [int(x) if idx == 0 else x for idx, x in enumerate(bigrams[i].split("\t"))]
    
    if bigrams[i][0] > 1:
        if bigrams[i][1]  in dictionary_prediction:
            dictionary_prediction[bigrams[i][1]].add((bigrams[i][0], bigrams[i][2]))
        else:
            dictionary_prediction[bigrams[i][1]] = {(bigrams[i][0], bigrams[i][2])}

In [40]:
def predict_next_word(bigram):
    possibles = []
    next_word_prediction = dictionary_prediction[bigram] if bigram in dictionary_prediction else dictionary_prediction["a"]
    for word in next_word_prediction:
        possibles.append(word[1])
    return possibles

In [41]:
with open("coca_all_links.txt", "r") as f:
    all_links = f.read()

dictionary = [word for word in word_tokenize(re.sub(r'[^\w\s]', '', all_links.lower()))]

In [42]:
def fix_misspelled(all_links):
    fixed_words = []
    previous_word = "a"
    words = word_tokenize(re.sub(r'[^\w\s]', '', all_links.lower()))

    for word in words:
        if word not in dictionary:
            candidates = predict_next_word(previous_word)
            ranked_candidates = sorted(candidates, key=lambda word_a: nltk.edit_distance(word, word_a))
            fix = ranked_candidates[0] if ranked_candidates else 0
            fixed_words.append(fix if fix else word)
        else:
            fixed_words.append(word)
        previous_word = word

    return ' '.join(fixed_words)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Explanation

First of all, it is required to learn whether the word is misspelled or right written. To identify this I've downloaded a file containing correctly spelled words. The original text is a book from norvig which is a book written by Doyle. It's an old book, so it can somehow reflect to the language. But generally it should works quite well. 

I chose the bigrams, because of the high sensitivity of 5-grams due to pattern consisting of 5 words. 

To solve it, I worked with a word precedes to the misspelled word. I thought that the following word is redundant, so I decided not to take it into consideration. Usually the correct word is already in the candidates' list, if not so, it is most probably neither in predecessor nor successor. Due to this my solution became much easier and faster. 

I sorted all the candidates gained by bigrams to choose the most appropriate one. To calculate the weights I took the distance between the words. I used nltk.edit_distance (which uses Levenshtein edit distance)

Sometimes situation can happen that no word is included in the birgram vocabulary, and exactly for this case I return candidates for the "a" due to it's being one of the most frequent words. And there are a lot of possible candidates for this word. 

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [44]:
def find_words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(find_words(all_links))

In [45]:
def edits_one(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits, deletes, transposes, replaces, inserts = [(word[:i], word[i:]) for i in range(len(word) + 1)], [], [], [], []

    for left, right in splits:
        if right:
            deletes.append(left + right[1:])
        if len(right) > 1:
            transposes.append(left + right[1] + right[0] + right[2:])
        for c in letters:
            if right:
                replaces.append(left + c + right[1:])
            inserts.append(left + c + right)
    
    return set(deletes + transposes + replaces + inserts)

def edits_two(word):
    all_edits = []
    for e1 in edits_one(word):
        for e2 in edits_one(e1):
            all_edits.append(e2)
    return set(all_edits)

In [46]:
def find_candidates(word):
    candidate_words = set()
    if word in WORDS:
        candidate_words.add(word)
    
    edits1_words = edits_one(word)
    for edit in edits1_words:
        if edit in WORDS:
            candidate_words.add(edit)
    
    edits2_words = edits_two(word)
    for edit in edits2_words:
        if edit in WORDS:
            candidate_words.add(edit)

    if not candidate_words:
        candidate_words.add(word)
    
    return candidate_words

In [47]:
def calculate_probability(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def fixing(word): 
    return max(find_candidates(word), key=calculate_probability)

In [48]:
def fix_misspelled_Norvig(text):
    fixed_words = []
    words = word_tokenize(re.sub(r'[^\w\s]', '', text.lower()))
    for word in words:
        if word not in dictionary:
            fixed = fixing(word)
            fixed_words.append(fixed if fixing else word)
        else:
            fixed_words.append(word)
    return ' '.join(fixed_words)


In [49]:
def add_noise(text, frequency):
    noisy_text = ""
    text = text.lower()
    for word in text:
        noisy_text += rd.choice(string.ascii_letters).lower() if (rd.randint(0, frequency) == 1 and word.isalpha()) else word
    return noisy_text


In [51]:
def get_score(text1, text2):
    score = 0
    text1, text2 = text1.split(), text2.split()
    length = len(text1)
    for x, y in zip(text1, text2):
        score += 1 if x == y else 0
    return(score / length)

In [52]:
sample_text = '''
I once used an activity-centered control, setting it to present my photographs
to the audience. All worked well until I was asked a question. I paused to answer
it, but wanted to raise the room lights so I could see the audience. No, the
activity of giving a talk with visually presented images meant that room lights
were fixed at a dim setting. When I tried to increase the light intensity, this took
me out of “giving a talk” activity, so I did get the light to where I wanted it, but
the projection screen also went up into the ceiling and the projector was turned
off. The difficulty with activity-based controllers is handling the exceptional
cases, the ones not thought about during design.
Activity-centered controls are the proper way to go, if the activities are
carefully selected to match actual requirements. But even in these cases, manual
controls will still be required because there will always be some new, unexpected
demand that requires idiosyncratic settings. As my example demonstrates,
invoking the manual settings should not cause the current activity to be canceled.'''

In [53]:
text_misspelled = add_noise(sample_text, 80)
my_corrected_text = fix_misspelled(text_misspelled)
norvig = fix_misspelled_Norvig(text_misspelled)
my_score = get_score(sample_text, my_corrected_text)
norvig_score = get_score(sample_text, norvig)

print(f"\033[1mEasy test \033[0m")
print(f"My score is: {my_score}")
print(f"Norvig's solution score: {norvig_score}")
print('\n')


text_misspelled = add_noise(sample_text, 60)
my_corrected_text = fix_misspelled(text_misspelled)
norvig = fix_misspelled_Norvig(text_misspelled)
my_score = get_score(sample_text, my_corrected_text)
norvig_score = get_score(sample_text, norvig)

print(f"\033[1mMedium test \033[0m")
print(f"My score is: {my_score}")
print(f"Norvig's solution score: {norvig_score}")
print('\n')


text_misspelled = add_noise(sample_text, 40)
my_correction = fix_misspelled(text_misspelled)
norvig = fix_misspelled_Norvig(text_misspelled)
my_score = get_score(sample_text, my_correction)
norvig_score = get_score(sample_text, norvig)

print(f"\033[1mHard test \033[0m")
print(f"My score is: {my_score}")
print(f"Norvig's solution score: {norvig_score}")
print('\n')


Easy test 
My score is: 0.7540983606557377
Norvig's solution score: 0.5901639344262295


Medium test 
My score is: 0.7595628415300546
Norvig's solution score: 0.5792349726775956


Hard test 
My score is: 0.7377049180327869
Norvig's solution score: 0.5628415300546448


